In [1]:
# Required modules

import os
import glob
import tqdm
import keras
import librosa
import numpy as np
import pandas as pd

from PIL import Image
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-04-12 14:38:01.273214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 14:38:01.480974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_

In [2]:
# Load the 30 sec data features

features30 = pd.read_csv("./Data/features_30_sec.csv")
features30.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
# Load the images

image_size = (432, 288)  # Desired image size (width, height)
context = list()
all_images = list()
labels = list()

print("Processing image files...")
for dir_name in os.listdir("./Data/images_original/"):
    print(f"Resizing images in '{dir_name}' category")
    for img_fname in tqdm.tqdm(glob.glob(f"./Data/images_original/{dir_name}/*.png")):  # Add other formats as needed
        fn = (f"{img_fname.split('/')[-1].split('.')[0][:-5]}.{img_fname.split('/')[-1].split('.')[0][-5:]}.wav")
        context.append(features30[features30["filename"] == fn].values[0][2:-1])
        try:
            img = Image.open(img_fname)
#             img_resized = img.resize(image_size, Image.ANTIALIAS)
            all_images.append(np.array(img))
            labels.append(dir_name)
        except Exception as e:
            print(f"Error resizing {img_fname}: {e}")
            
all_images, context, labels = np.array(all_images).astype('float32'), np.array(context).astype('float32'), np.array(labels)

Processing image files...
Resizing images in 'blues' category


100%|██████████| 100/100 [00:00<00:00, 186.78it/s]


Resizing images in 'classical' category


100%|██████████| 100/100 [00:00<00:00, 267.67it/s]


Resizing images in 'country' category


...

100%|██████████| 100/100 [00:00<00:00, 252.52it/s]


Resizing images in 'reggae' category


100%|██████████| 100/100 [00:00<00:00, 263.32it/s]


Resizing images in 'rock' category


100%|██████████| 100/100 [00:00<00:00, 202.98it/s]


In [4]:
# Encode the labels

encoder = LabelEncoder()
labels = encoder.fit_transform(labels).astype('float32')

## CNNS

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, concatenate
from tensorflow.keras.layers import BatchNormalization

# Image branch
input_image = Input(shape=(288, 432, 4))
x1 = Conv2D(32, (3, 3), activation='relu')(input_image)
x1 = MaxPooling2D(2, 2)(x1)
x1 = Conv2D(64, (3, 3), activation='relu')(x1)
x1 = MaxPooling2D(2, 2)(x1)
x1 = Conv2D(128, (3, 3), activation='relu')(x1)
x1 = MaxPooling2D(2, 2)(x1)
x1 = Flatten()(x1)

# Additional features branch
input_features = Input(shape=(57,))
x2 = Dense(128, activation='relu')(input_features)
x2 = Dropout(0.5)(x2)
x2 = BatchNormalization()(x2)  # Optional, helps to stabilize learning

# Merge branches
combined = concatenate([x1, x2])

# Fully connected layers
x = Dense(512, activation='relu')(combined)
x = Dropout(0.5)(x)
output = Dense(1, activation='softmax')(x)  # Assuming 10 classes

# Create the model
model = Model(inputs=[input_image, input_features], outputs=output)

2024-04-12 14:38:11.844032: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/openmpi-4.1.2-4a/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/tcl-8.6.11-d4/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/sqlite-3.37.1-6s/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/zlib-1.2.11-2y/lib:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib64:/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-9.3.0/gcc-10.3.0-ya/lib
2024-04-12 14:38:11.844101: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-12 14:38:11.844126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel drive

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    x=[all_images, pd.DataFrame(context)],
    y=labels,
    epochs=50,
    validation_split=0.2,
)

Epoch 1/50
18/25 [====================>.........] - ETA: 16s - loss: 0.0000e+00 - accuracy: 0.1233

In [ ]:
# Evaluate the model
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation loss: {val_loss}, Validation accuracy: {val_acc}')

# Save the model
model.save('path/to/save/model.h5')